# Time Series Regression Analysis (Corporation Favorita)

## `Business Understanding`
**Project Scenario**

You are a data scientist in Corporation Favorita, a large Ecuadorian-based grocery retailer. Corporation Favorita wants to ensure that they always have the right quantity of products in stock. To do this you have decided to build a series of machine learning models to forecast the demand of products in various locations. The marketing and sales team have provided you with some data to aid this endeavor. Your team uses CRISP-DM Framework for Data Science projects

**Problem Statement**

Corporation Favorita aims to optimize its inventory management by accurately forecasting the demand for various products across its stores in Ecuador. The goal is to ensure that each store has the right quantity of products in stock to meet customer demand while minimizing overstocking or stockouts
    
**Objective**

The objective is to build machine learning models that can predict unit sales for different product families at Favorita stores accurately. These models will help optimize inventory levels, improve sales forecasting accuracy, and ultimately enhance customer satisfaction by ensuring product availability.

**Key Stakeholders**

Stakeholders include Corporation Favorita's management, sales and marketing teams, store managers, and data science team.

**Analytical Goals**
- Handle missing values in the datasets by imputation techniques such as mean, median, or mode.
- Address outliers in sales data that may skew the model's predictions by applying robust statistical methods or trimming techniques.
- Normalize or scale numerical features to ensure uniformity and improve model performance.
- Encode categorical variables using techniques such as one-hot encoding or label encoding.
- Build time series regression models such as SARIMA, ARIMA, XGBoost, Linear Regression etc. to capture seasonality and trends in sales data.
- Validate models using cross-validation techniques and assess their performance metrics such as RMSE (Root Mean Squared Error) or MAE (Mean Absolute Error).
- Create insightful visualizations and dashboards for sales analysis and forecasting.

**Success Criteria**
- Achieve a 0.2 RMSE (Root Mean Squared Error) in sales forecasting models.
- Improve inventory management efficiency and reduce stockout instances.

**Constraints and Assumptions**
- Assumption: Historical sales data is representative of future demand patterns.
- Constraint: Limited availability of real-time sales data for model training.

**Data Requirements**
- Utilize data from train.csv, stores.csv, holidays_events.csv, oil.csv, and transaction.csv for analysis.
- Include features such as store_nbr, family, onpromotion, store metadata, oil prices, holidays, and transactional data.

**Business Impact**
- Enhance customer satisfaction through better product availability.
- Optimize inventory management, leading to cost savings and improved operational efficiency.

### `Hypothesis`
Null Hypothesis (Ho): Holidays do not have a significant effect on the sales 

Alternate Hypothesis (Ha): Holidays have a significant effect on the sales

**Analytical Business Questions**

1. Is the train dataset complete (has all the required dates)?
2. Which dates have the lowest and highest sales for each year (excluding days the store was closed)?
3. Compare the sales for each month across the years and determine which month of which year had the highest sales.
4. Did the earthquake impact sales?
5. Are certain stores or groups of stores selling more products? (Cluster, city, state, type)
6. Are sales affected by promotions, oil prices and holidays?
7. What analysis can we get from the date and its extractable features?
8. Which product family and stores did the promotions affect.
9. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)
10. Does the payment of wages in the public sector on the 15th and last days of the month influence the store sales.


Import Libraries

In [ ]:
# Import Data Manupilation Packages
from dotenv import dotenv_values
import pyodbc
import pandas as pd
import polars as pl
import numpy as np

# Import Visualization packages
import seaborn as sns
import seaborn_polars as snl
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.polars # noqa

# Import stats packages
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

from itertools import product

# Utility Packages:
pd.options.display.float_format = '{:.2f}'.format
import warnings
warnings.filterwarnings('ignore')

print("🛬 Imported all packages.", "Warnings hidden. 👻")

## `Data Understanding`

#### `First Datasets from Database`
I'm using Python's dotenv with a .env file to safely fetch the first datasets from a SQL database into my notebook. This keeps my database credentials private while allowing easy access to the data for analysis.

In [ ]:
# Load environment variables
environment_variables = dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

In [ ]:
# Create a connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

In [ ]:
# Use the connect method of pyodbc library and pass in the connection string
connection = pyodbc.connect(connection_string)

In [ ]:
# sql query to get oil data
query1 = "SELECT * FROM dbo.oil"
oil_df = pl.read_database(query1, connection)

# sql query to get holidays_events
query2 = "SELECT * FROM dbo.holidays_events"
holiday_events_df =  pl.read_database(query2, connection)

#sql query to get stores
query3 = "SELECT * FROM dbo.stores"
stores_df =  pl.read_database(query3, connection)

In [ ]:
oil_df.head()

In [ ]:
def to_date(df, date_col):
    # Convert date column to date type
    df = df.with_columns(df[date_col].cast(pl.Date))
    
    # Sort the dataframe using the date column
    df = df.sort(date_col)
        
    return df

In [ ]:
# Convert date column to date type
oil_df = to_date(oil_df, 'date')
oil_df

In [ ]:
oil_df.glimpse()

In [ ]:
oil_df.null_count()

In [ ]:
holiday_events_df.head()

In [ ]:
# Convert date column to datetime type
holiday_events_df = to_date(holiday_events_df, 'date')
holiday_events_df

In [ ]:
holiday_events_df.glimpse()

In [ ]:
holiday_events_df.null_count()

In [ ]:
stores_df.head()

In [ ]:
stores_df.glimpse()

In [ ]:
stores_df.null_count()

`Second Datasets from Github`

I obtained the second datasets from a GitHub repository, and I'll use Pandas to import the CSV file into my notebook for analysis.

In [ ]:
# Loading the train Data
train_df = pl.read_csv('Data/train.csv')

# Loading the transaction data
transactions_df = pl.read_csv('Data/transactions.csv')

In [ ]:
transactions_df.head(3)

In [ ]:
# Convert date column to date type
transactions_df = to_date(transactions_df, 'date')
transactions_df

In [ ]:
transactions_df.glimpse()

In [ ]:
train_df.null_count()

In [ ]:
train_df.head(3)

In [ ]:
train_df.glimpse()

In [ ]:
# Convert date column in train df to date type
train_df = to_date(train_df, 'date')

In [ ]:
# Preview of the train data frame
train_df.head()

In [ ]:
train_df.glimpse()

In [ ]:
train_df.null_count()

### **Exploratory Data Analysis (EDA)**

In [ ]:
# Checking the descriptive statistics of the train data set
train_df.describe()

In [ ]:
# Checking  for unique values in all the columns
cols = train_df.columns

results = []

for col in cols:
    
    unique_values = train_df[col].unique()
    num_unique_values = train_df[col].n_unique()
    results.append([col, unique_values, num_unique_values])

results_df = pl.DataFrame(results, schema=['Column', 'Unique_Values', 'Num_Unique_Values'])
results_df

Checking for Data Completness and Missing dates in our Date column

In [ ]:
# Start of the data using the date 
start_date = train_df['date'].min()

# End of the data using the date
end_date = train_df['date'].max()

# Print the start and end date of the data
print(f'Start date is {start_date}')
print(f'End date is {end_date}')

In [ ]:
def missing_dates(df):
    # Getting the date range for the train data set
    date_range = pl.date_range(start = start_date, end = end_date, interval = '1d', eager = True)
    
    # Getting the existing dates
    existing_dates = df['date']

    # Getting the missing dates using the date range and the existing dates
    missing_dates = date_range.filter(~date_range.is_in(existing_dates))
    
    return missing_dates

- Check missing dates

In [ ]:
# Check missing dates
all_missing_dates = missing_dates(train_df)
all_missing_dates

In [ ]:
# Dropping the Id column since it will not be relevant for our visualizations
train_df = train_df.drop(columns = ['id'])

In [ ]:
train_df.shape

In [ ]:
# Define the columns to fill
columns = [column for column in train_df.columns if column != 'date']
columns

In [ ]:
no_enteries_per_day = (train_df.select(pl.col('date').filter(date=pl.col('date').min()))).shape[0]
no_enteries_per_day

In [ ]:
no_enteries_per_day * len(all_missing_dates)

- Create missing dataframe using all missing dates, unique store_nbr and unique family category

In [ ]:
# Create df with unique values for store_nbr
store_nbr_unique = train_df['store_nbr'].unique()

# Create df with unique values for family category
family_unique = train_df['family'].unique()

# Create dataframe with missinng dates and unique store_nbr and family category
missing_df = pl.DataFrame(
    list(product(all_missing_dates, store_nbr_unique, family_unique)), 
    schema=['date', 'store_nbr', 'family']
)

missing_df.shape


- Add sales and onpromotion columns to the missing_df

In [ ]:
missing_df = missing_df.with_columns(
    sales=None,
    onpromotion=None,
)

missing_df.glimpse()

- Join missing_df to the train_df

In [ ]:
# Join original train_df with missing_df
train_df = train_df.vstack(missing_df).sort('date')
train_df.glimpse()


In [ ]:
train_df.null_count()

- Again, check missing dates

In [ ]:
# Check missing dates
all_missing_dates = missing_dates(train_df)
all_missing_dates

In [ ]:
train_df.null_count()

In [ ]:
train_df.glimpse()

- Fill missing values in sales with 0.0 and onpromotion with 0

In [ ]:
# Fill sales with zero strategy
train_df = train_df.fill_null(strategy="zero")

train_df.glimpse()

- Now, there are no missing values in sales and onpromotion columns

In [ ]:
train_df.null_count()

**Feature Engineering**

In [ ]:
# Extracting the year from the date and adding it to the df as a new column
train_df = train_df.with_columns(pl.col('date').dt.year().alias('year'))

# Extracting the month from the date and adding it to the df as a new column
train_df = train_df.with_columns(pl.col('date').dt.month().alias('month'))

# Extracting the day from the date and adding it to the df as a new column
train_df = train_df.with_columns(pl.col('date').dt.day().alias('day'))

# Preview of the train Data frame
train_df.glimpse()

### **Univariate Analysis**

In [ ]:
train_df.glimpse()

In [ ]:
train_df.drop('family').plot()

In [ ]:
plt.figure(figsize = (15,5))
snl.boxplot(train_df.drop(['year', 'sales', 'onpromotion', 'family']), orient = 'h')
plt.show()

In [ ]:
# Select columns to keep for the boxplot
columns_to_keep = [col for col in train_df.columns if col not in ['year', 'sales', 'onpromotion', 'family']]
df_for_boxplot = train_df.select(columns_to_keep)

# Plotting the boxplot
plt.figure(figsize=(12, 5))
snl.boxplot(df_for_boxplot, orient='h')
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
snl.boxplot(train_df['year'], orient = 'h')
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
snl.boxplot(train_df['onpromotion'], orient = 'h')
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
snl.boxplot(train_df['sales'], orient = 'h')
plt.show()

In [ ]:
train_df.columns

In [ ]:
train_df.head()

In [ ]:
plt.figure(figsize=(15, 5))
snl.kdeplot(train_df.drop(columns=['year', 'sales']), alpha=0.3, fill=True)
plt.title('Density Plot for Features')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
snl.kdeplot(train_df['year'], alpha=0.3, fill=True)
plt.title('Density Plot for year')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
snl.kdeplot(train_df['sales'], alpha=0.3, fill=True)
plt.title('Density Plot for Sales')
plt.show()

In [ ]:
sales_skew = train_df['sales'].skew()
onpromotion_skew = train_df['onpromotion'].skew()
store_skew = train_df['store_nbr'].skew()
date_skew = train_df['date'].skew()

print(f'Sales skewness = {sales_skew}')
print(f'onpromotion skewness = {onpromotion_skew}')
print(f'Store skewness = {store_skew}')
print(f'Date skewness = {date_skew}')

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size
plt.plot(train_df['date'], train_df['sales'], marker='o', linestyle='-', color='b')  # Plotting sales against date
plt.xlabel('Date')  # Label for the x-axis
plt.ylabel('Sales')  # Label for the y-axis
plt.title('Sales Over Time')  # Title for the plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True)  # Enable grid lines
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()  # Show the plot

In [ ]:
plt.figure(figsize=(16, 4)) 
sns.lineplot(train_df, x=train_df['date'], y=train_df['sales'])
plt.title(f'Daily sales over time')
plt.show()

## **Bi-Variate Analysis**

Exploring relationships

In [ ]:
train_df_numeric = train_df.select(pl.selectors.by_dtype(pl.Date, pl.INTEGER_DTYPES, pl.FLOAT_DTYPES))

# Getting the correlation of numeric values in the train dataset
correlation = train_df_numeric.corr()

# Create the heatmap using Plotly Express
fig = px.imshow(correlation,
                labels=dict(color="Correlation"),
                x=correlation.columns,
                y=correlation.columns,
                text_auto=True,  # Automatically add text in each cell
)

# Update layout for better readability
fig.update_layout(title="Correlation Heatmap", height=700)

# Set the size of text annotations
fig.update_traces(
    text=correlation.to_numpy(),
    texttemplate="%{text:.4f}",
    textfont_size=12
)

fig.show()

In [ ]:
# Visualize relationships 
plt.scatter(train_df['onpromotion'], train_df['sales'])
plt.xlabel('On Promotion')
plt.ylabel('Sales')
plt.title('Sales vs On Promotion')
plt.show()

Understand relationship on sales with external factors

In [ ]:
# Merge dataframes to analyze relationships with external factors
merged_df = train_df.join(holiday_events_df, on='date', how='left')
merged_df = merged_df.join(oil_df, on='date', how='left')
merged_df = merged_df.join(transactions_df, on=['date', 'store_nbr'], how='left')

# Analyze relationships with external factors 
sales_by_holiday = merged_df.group_by('type').agg(pl.col('sales').sum())
print("Sales by Holiday Type:")
print(sales_by_holiday)

Decompostion

In [ ]:
# Using additive model, calculate the additive results for seasons, trends and noise for the sales
additive_results = seasonal_decompose(train_df['sales'].to_pandas(), model = 'additive', period = 365)

# Update rcParams to set the figure size for the plot
plt.rcParams.update({'figure.figsize': (15, 5)})

# Plot the decomposed components (trend, seasonal, residual)
additive_results.plot()

# Display the plot
plt.show()

Time Series Decomposition:

In [ ]:
# Perform time series decomposition
decomposition = seasonal_decompose(train_df['sales'].to_pandas(), model='additive', period=12)

# Plot decomposition components
plt.figure(figsize=(12, 8))
plt.subplot(411)
plt.plot(decomposition.observed)
plt.subplot(412)
plt.plot(decomposition.trend)
plt.subplot(413)
plt.plot(decomposition.seasonal)
plt.subplot(414)
plt.plot(decomposition.resid)
plt.tight_layout()
plt.show()


Autocorrelation and Partial Autocorrelation:

In [ ]:
# Plot autocorrelation and partial autocorrelation
plt.figure(figsize=(12, 4))
plot_acf(train_df['sales'], lags=40, alpha=0.05)
plt.title('Autocorrelation')
plt.show()

plt.figure(figsize=(12, 4))
plot_pacf(train_df['sales'], lags=40, alpha=0.05)
plt.title('Partial Autocorrelation')
plt.show()


Stationarity Test and Differencing

In [ ]:
# Perform Augmented Dickey-Fuller test for stationarity
result = adfuller(train_df['sales'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])

# Perform differencing if needed for stationarity
if result[1] > 0.05:
    differenced_sales = train_df['sales'].diff().dropna()
else:
    differenced_sales = train_df['sales']


Seasonal Analysis

In [ ]:
decomposition.seasonal

In [ ]:
# Calculate seasonal indices
seasonal_indices = decomposition.group_by('seasonal')(decomposition.seasonal.index.month).mean()

# Plot seasonal indices
plt.figure(figsize=(8, 4))
plt.plot(seasonal_indices)
plt.xlabel('Month')
plt.ylabel('Seasonal Index')
plt.title('Seasonal Indices')
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()


Event Analysis

In [ ]:
# Merge holiday/event data with sales data
merged_df = train_df.join(holiday_events_df, on='date', how='left')

# Plot sales during holidays/events
plt.figure(figsize=(12, 6))
plt.plot(merged_df['date'], merged_df['sales'], label='Sales')
plt.plot(merged_df.filter(pl.col('type') == 'Holiday')['date'], merged_df.filter(pl.col('type') == 'Holiday')['sales'], 'ro', label='Holidays')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Sales During Holidays/Events')
plt.legend()
plt.show()


**Key Insights**
1. We have identified missing dates in our date column, so we need to fill them in.
2. Our data is not on the same scale after reviewing the summary statistics. 
3. Our train dataset has a positive skewness
6. Noticed seasonal patterns in sales data, especially during holidays and events.
7. Identified outliers in sales, on promotion data that may need further investigation.
